In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json

## Grab some books from THR

In [30]:
def getBook(id):
    resp = requests.get('http://tarheelreader.org/book-as-json?id=%d' % id)
    return resp.json()

def findBooks(page):
    resp = requests.get('http://tarheelreader.org/find/?json=1&page=%d' % page)
    return [book['ID'] for book in resp.json()['books']]

books = []
for i in range(1, 4):
    books.extend([ getBook(i) for i in findBooks(i) ])

## Write them to the new instance

In [31]:
passwd = '2d46 RG2n wzZ2 c4re'
userid = 'admin'

# 2016-08-04 15:59:38

from datetime import datetime
from pprint import pprint

def createBook(o):
    b = {}
    b['title'] = o['title']
    b['content'] = [ 
        {
        'caption': page['text'],
        'image': page['url'] 
        } for page in o['pages'] ]
    b['pseudonym'] = o['author']
    b['language'] = o['language']
    b['date'] = datetime.strptime(o['created'], "%Y-%m-%d %H:%M:%S").isoformat()
    b['modified'] = datetime.strptime(o['modified'], "%Y-%m-%d %H:%M:%S").isoformat()
    b['categories'] = [3]
    b['rating_count'] = o['rating_count']
    b['rating_sum'] = o['rating_total']
    b['reviewed'] = o['reviewed']
    b['status'] = 'publish'
    b['comment_status'] = 'closed'
    b['ping_status'] = 'closed'
    return b
    
for book in books:
    b = createBook(book)
    resp = requests.post('http://localhost:8000/wp-json/wp/v2/posts', json=b, 
                         auth=HTTPBasicAuth(userid, passwd))
    